# **AI Research Assistant**

AI assitant that discovers, filters, and analyzes web content using Crawö4AI's URL Seeder to:

* Discover all available URLs without crawling them first.
* Score and rank them by relevance using AI
* Crawl only the most relevant content
* Generate research insights with proper citations.

**About the research assistant** :

A smart research assistant that:

1. Takes any research query (eg. Knowledge graphs)
2. Discovers relevant articles from news sites
3. Ranks them by relevance using BM25 scoring
4. Crawls only the top-ranked articles
5. Synthesizes findings into a comprehensive report
   

## Pipeline Overview

User Query -> Query Enhancement -> URL Discovery -> Relevance Scoring -> Smart Crawling -> AI Synthesis. -> Research Report

In [1]:
import asyncio
import json
import os
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass, asdict
from datetime import datetime
from pathlib import Path

#Rich for beutiful console output
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
from rich.progress import Progress, SpinnerColumn, TextColumn

# Crawl4AI imports for intelligent crawling
from crawl4ai import (
    AsyncWebCrawler,
    BrowserConfig,
    CrawlerRunConfig,
    AsyncUrlSeeder,
    SeedingConfig,
    AsyncLogger,
    PruningContentFilter,
    DefaultMarkdownGenerator
)


# LiteLLM for AI capabilities
import litellm

# Initialize Rich console for pretty outputs
console=Console()

print("Environment ready :) All dependencies loaded successfully.")

Environment ready :) All dependencies loaded successfully.


## Step 1: Configuration and Data Classes

Here we define the research pipeline configuration. These dataclasses act as out control center, allowing us to fine-tune every aspect of the research process. Think of them as the settings panel for the research assistant, from discovery limits to AI model choices.


In [2]:
@dataclass
class ResearchConfig:
        """
        Configuration for the research pipeline
        
        This class controls every aspect of our research assistant:
        - How many URLs to discover and crawl
        - Which scoring methods to use
        - Whether to use AI enhancement
        - Output preferences
        """
        
        # Core Settings
        domain: str= "www.bbc.com/sport"
        max_urls_discovery: int =100    # Cast a wide net initially
        max_urls_to_crawl: int=10       # But only crawl the best
        top_k_urls: int=10              # Focus on top results
        
        # Scoring and filtering
        score_threshold: float=0.3      # Minimum relevance score
        scoring_method: str="bm25"      # BM25 is great for relevance
        
        # AI and processing
        use_llm_enhancement: bool=True  # Enhance queries with AI
        llm_model: str="openai/gpt-4o-mini" # Fast and capable
        
        # URL discovery options
        extract_head_metada: bool = False   # Get titles, descriptions
        live_check: bool= True              # Verify URLs are accessible
        force_refresh: bool= True           # Bypass cache
        
        # Crawler settings
        max_concurrent_crawls: int=5        # Parallel crawling
        timeout: int = 30000                # 30 second timeout
        headless: bool = True               # No browser window
        
        # Output settings
        output_dir: Path = Path("research_results")
        verbose: bool=True

@dataclass
class ResearchQuery:
    """Container for research query and metadata """
    original_query: str
    enhanced_query: Optional[str] = None
    search_patterns: List[str] = None
    timestamp: str = None
    
@dataclass
class ResearchResult:
    """Container for research results"""
    query: ResearchQuery
    discovered_urls: List[Dict]
    crawled_content: List[Dict]
    synthesis: str
    citations: List[Dict]
    metadata: Dict

# Create default configuration
config= ResearchConfig()
console.print(Panel(
    f"[bold cyan]Research Configuration[/bold cyan]\n"
    f" Domain: {config.domain}\n"
    f" Max Discovery: {config.max_urls_discovery}\n"
    f" Max Crawl: {config.max_urls_to_crawl}\n"
    f" AI Model: {config.llm_model}",
    title="Settings"
))

    

╭─────────────────────────────────────────────────── Settings ────────────────────────────────────────────────────╮
│ Research Configuration                                                                                          │
│  Domain: www.bbc.com/sport                                                                                      │
│  Max Discovery: 100                                                                                             │
│  Max Crawl: 10                                                                                                  │
│  AI Model: openai/gpt-4o-mini                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Step 2: Query Enhancement with AI

Not all search queries are created equal. Here we use AI to transform simple queries into comprehensive search strategies. The LLM analyzes your query, extracts key concepts, and generates related terms - turning "football news" into a rich set of search patters.

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

In [4]:
async def enhance_query_with_llm(query: str,config:ResearchConfig) -> ResearchQuery:
    """
    Transform simple queries into comprehensive search strategies
    Why enhance queries?
    - User often use simple terms ("football news")
    - But relevant content might use varied terminology
    - AI helps capture all relevant variations    
    """
    
    console.print(f"\n[cyan] Enhancing query: '{query}...[/cyan]")
    try:
        # Ask AI to analyze and expand the query
        response = await litellm.acompletion(
            model=config.llm_model,
            messages=[{
                "role":"user",
                "content":f"""Given this research query: "{query}"
                Extract:
                1. Key terms and concepts (as a list)
                2. Related search terms
                3. A more specific/enhanced version of the query
                
                Return as JSON:
                {{
                    "key_terms":["term1","term2"],
                    "related_terms": ["related1","related2"],
                    "enhanced_query": "enhanced version of query"
                }}
                
               """
            }],
            temperature=0.3, #Low termperature for consistency
            response_format={"type":"json_object"}
        )
        
        data=json.loads(response.choices[0].message.content)
        
        # Create search patterns from extracted terms
        # These patterns help the URL seeder find relevant pages
        
        all_terms= data["key_terms"] + data ["related_terms"]
        #patterns = [f"*{term.lower()}*" for term in all_terms]
        
        result = ResearchQuery(
            original_query=query,
            enhanced_query=data["enhanced_query"],
            search_patterns= "", #patterns[:10], #Limit to 10 patterns
            timestamp=datetime.now().isoformat()
        )
        
        # Show the enhancement
        console.print(Panel(
            f"[green] Enhanced Query:[/green] {result.enhanced_query}\n"
            f"[dim] Key terms: {', '.join(data['key_terms'])}[/dim]",
            title = "Query Enhancement"
        ))
    
        return result

    except Exception as e:
        console.print(f"[yellow] Enhancement failed, using original query: {e}[/yellow]")
        #Fallback to simple tokenization
        words= query.lower().split()
        patterns =[f"*{word}*" for word in words if len(word)>2]
        
        return ResearchQuery(
            original_query=query,
            enhanced_query=query,
            search_patterns=patterns,
            timestamp=datetime.now().isoformat()
        )
        
# Example usage
test_query= "knowledge graphs news"
enhanced = await enhance_query_with_llm(test_query,config)

 Enhancing query: 'knowledge graphs news...

╭─────────────────────────────────────────────── Query Enhancement ───────────────────────────────────────────────╮
│  Enhanced Query: latest advancements and applications of knowledge graphs in news and media                     │
│  Key terms: knowledge graphs, semantic web, data integration, ontology, graph databases, machine learning,      │
│ natural language processing                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Step 3: Smart URL Discovery with AsyncUrlSeeder

This is where the magic begins! Instead of crawling pages to find links, AsyncUrlSeeder discovers URLs from sitemaps and Common Crawl data. It's like having a map of the entire website before you start exploring. We'll discover hundreds of URLs in seconds, complete with metadata.

In [11]:
async def discover_urls(
    domain: str,
    query: ResearchQuery,
    config: ResearchConfig
 ) -> List[Dict]:
   """
   Discover and rank URLs without crawling them
    
   The URL Seeder is incredibly powerful because it:
   1. Gets URLs from sitemaps (official site maps)
   2. Get URLs from Common Crawl (web-scale data)
   3. Extracts metadata without full page loads
   4. Scores relevance using BM25 algorithm
    
   This means we know which pages are worth crawling
   BEFORE we spend time crawling them!
   """
   console.print(f"\n[cyan] Discovering URLs from {domain}...[/cyan]")
    
   # Use context manager for automatic cleanup
   async with AsyncUrlSeeder(logger=AsyncLogger(verbose=config.verbose)) as seeder:
      # Configure the discovery process
      seeding_config = SeedingConfig (
         # Data sources
         source = "sitemap", # Use both sitemap AND Common Crawl # REVIEW THIS URGENT
         
         # Metadata extraction
         extract_head=config.extract_head_metada, #Get titles, descriptions
         
         # Relevance scoring
         query=query.enhanced_query or query.original_query,
         scoring_method=config.scoring_method, #BM25 scoring
         score_threshold=config.score_threshold, # Minimum scoring
         
         # Limits and performance
         max_urls= config.max_urls_discovery,
         live_check=config.live_check,
         force=config.force_refresh, #Bypass cache if needed
         
         # Performance tuning
         # Concurrency = 1000, # Parallel workers
      ) 
      try:
         # Discover URLs - this is fast
         urls=await seeder.urls(domain, seeding_config)
          
         # Results are already sorted by relevance
         # thanks to BM25 scoring
         top_urls = urls[:config.top_k_urls]
          
         # Show discovery results
         console.print(f"[green] Discovered {len(urls)} URLs, selected top {len(top_urls)}[/green]")
          
         # Display a sample of what we found
         if top_urls:
            table= Table(title ="Top Discovered URLs")
            table.add_column("Score", style="cyan")
            table.add_column("Title", style="green")
            table.add_column("URL",style="dim")
             
            for url in top_urls[:5]:
               score= f"{url.get('relevance_score',0):.3f}"
               title ="N/A"
               if url.get('head_data') and url['head_data'].get('title'):
                     title=url['head_data']['title'][:50] + "..."
               url_str = url['url'][:60] + "..."
               table.add_row(score, title, url_str)
            console.print(table)
         return top_urls
         
      except Exception as e:
         console.print(f"[red] URL discovery failed: {e}[/red]")
         return []
      
# Example discovery
discovered = await discover_urls(config.domain, enhanced, config) 
                
       

 Discovering URLs from www.bbc.com/sport...

[URL_SEED] ℹ Loading latest CC index from cache: 
/Users/fernandacladeramelgar/.crawl4ai/seeder_cache/latest_cc_index.txt 

[URL_SEED] ℹ Starting URL seeding for www.bbc.com/sport with source=sitemap 

[URL_SEED] ℹ Fetching from sitemaps... 

[URL_SEED] ℹ Found sitemap at https://www.bbc.com/sport/sitemap.xml 

...

[URL_SEED] ℹ Finished URL seeding for www.bbc.com/sport. Total URLs: 43 

[URL_SEED] ℹ Query provided but extract_head is False. Enable extract_head for relevance scoring. 

 Discovered 43 URLs, selected top 10

                    Top Discovered URLs                     
┏━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Score ┃ Title ┃ URL                                      ┃
┡━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 0.000 │ N/A   │ https://www.bbc.com/sport...             │
│ 0.000 │ N/A   │ https://www.bbc.com/sport/football...    │
│ 0.000 │ N/A   │ https://www.bbc.com/sport/formula1...    │
│ 0.000 │ N/A   │ https://www.bbc.com/sport/cricket...     │
│ 0.000 │ N/A   │ https://www.bbc.com/sport/rugby-union... │
└───────┴───────┴──────────────────────────────────────────┘

[URL_SEED] ℹ Closed HTTP client 

## Step 4: Intelligent Content Crawling

Now we crawl only the most relevant URLs. This is where our smart filtering pays off, instead of crawling hundreds of pages, we focus on the top 10-20 most relevant ones. We use content filtering to extract only the meaningful text, removing ads and navigation.

In [ ]:
async def crawl_selected_urls(
    urls:List[Dict],
    query: ResearchQuery,
    config: ResearchConfig
) -> List[Dict]:
    """
    Crawl only the most relevant URLs with smart content filtering
    Key optimizations:
    1. We already know these URLs are relevant (from scoring)
    2. We crawl them in parallel for speed
    3. We extract only meaningful content (no ads/nav)
    4. We generate clean markdown for analysis
    """
    
    # Extract URLs from discovery results
    url_list=[u['url'] for u in urls if 'url' in u][:config.max_urls_to_crawl]
    if not url_list:
        console.print("[red] No URLs to crawl [/red]")
        return []

    console.print(f"\n [cyan] Crawling {len(url_list)} URLs...[/cyan]")
    
    # Configure intelligent content extraction
    # This removes ads, navigation and other noise
    md_generator = DefaultMarkdownGenerator(
        content_filter = PruningContentFilter(
            threshold=0.48, # Content relevance threshold
            threshold_type ="dynamic", # Adapts to page structure
            min_word_threshold= 10 # Ignore tiny text blocks
        ),
    )
    # Configure the crawler
    crawler_config=CrawlerRunConfig(
        markdown_generator=md_generator,
        exclude_external_links=True, #Focus on content, not links
        excluded_tags= ['nav','header','footer','aside'], #Skip UI elements
    )
    
    # Create crawler with browser config
    async with AsyncWebCrawler(
        config=BrowserConfig(
            headless=config.headless,
            verbose=config.verbose
        )
    ) as crawler:
        # Crawl URLs in parallel for speed
        # arun_many handles concurrency automatically
        results = await crawler.arun_many(
            url_list,
            config=crawler_config,
            max_concurrent=config.max_concurrent_crawls
        )
        
        # Process successful results
        crawled_content=[]
        for url, result in zip(url_list,results):
            if result.success:
                # Extract the content we need
                content_data={
                    'url':url,
                    'title':result.metadata.get('title', 'No title'),
                    'markdown':result.markdown.fit_markdown or result.markdown.raw_mardown,
                    'metadata': result.metadata
                }
                crawled_content.append(content_data)
                console.print(f"[green] check [/green] Crawled: {url[:60]}...")
            else:
                console.print(f"[red] No check [/red] Failed: {url[:50]}... - {result.error}")
        console.print(f"[green] Successfully crawled {len(crawled_content)} pages[/green]")
        return crawled_content
    

# Example crawling
crawled = await crawl_selected_urls(discovered[:5], enhanced, config)